# CS 441 Final Project - Disaster Tweets

## 1. Import

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [25]:
# Load the dataset
train_df = pd.read_csv("train.csv")

# Split the dataset (80% train, 20% validation)
train_data, val_data = train_test_split(train_df, test_size = 0.2, random_state = 441)

print("Training set:", train_data.shape)
print("Validation set:", val_data.shape)

print("Example:\n", train_df.iloc[32])

Training set: (6090, 5)
Validation set: (1523, 5)
Example:
 id                                                         49
keyword                                                ablaze
location                        Est. September 2012 - Bristol
text        We always try to bring the heavy. #metal #RT h...
target                                                      0
Name: 32, dtype: object
